# FPU模型的分子动力学模拟
FPU问题的一般模型为
\begin{equation}
    F = k\Delta x + \alpha \Delta {x^2} + \beta \Delta {x^3}
\end{equation}
本文档使用Verlet算法进行模拟，假设无量纲化后
\begin{equation}
    {\ddot x_n} = k\left( {{x_{n + 1}} - {x_n}} \right) + \alpha {\left( {{x_{n + 1}} - {x_n}} \right)^2} + \beta {\left( {{x_{n + 1}} - {x_n}} \right)^3} - k\left( {{x_n} - {x_{n - 1}}} \right) - \alpha {\left( {{x_n} - {x_{n - 1}}} \right)^2} - \beta {\left( {{x_n} - {x_{n - 1}}} \right)^3}
\end{equation}

In [6]:
import numpy as np
from typing import Sequence
import matplotlib.pyplot as plt

In [7]:
def FPU(
    N: int,
    k: int | float,
    alpha: int | float,
    beta: int | float,
    r: Sequence,
    v_0: Sequence,
    dt: int | float = 0.01,
    step: int = 1000,
    method: str = "Verlet",
    is_ring: bool = False,
):
    # *FPU模型的分子动力学模拟
    # *N：粒子数，k,alpha,beta力中的参数
    # *r：初始位置，v_0:初始速度
    # *dt：时间步长，step：迭代次数
    # *method：方法，is_ring：粒子是否成环
    # TODO更多算法
    r_k = []
    v_k = []
    match method:
        case "Verlet":
            r_0 = [r[i] - v_0[i] * dt for i in range(N)]
            for i in range(step):
                if is_ring == True:
                    a = [
                        k * (r[i + 1] - r[i])
                        + alpha * (r[i + 1] - r[i]) ** 2
                        + beta * (r[i + 1] - r[i]) ** 3
                        - k * (r[i] - r[N - 1])
                        - alpha * (r[i] - r[N - 1]) ** 2
                        - beta * (r[i] - r[N - 1]) ** 3
                        if i == 0
                        else k * (r[0] - r[i])
                        + alpha * (r[0] - r[i]) ** 2
                        + beta * (r[0] - r[i]) ** 3
                        - k * (r[i] - r[i - 1])
                        - alpha * (r[i] - r[i - 1]) ** 2
                        - beta * (r[i] - r[i - 1]) ** 3
                        if i == N - 1
                        else k * (r[i + 1] - r[i])
                        + alpha * (r[i + 1] - r[i]) ** 2
                        + beta * (r[i + 1] - r[i]) ** 3
                        - k * (r[i] - r[i - 1])
                        - alpha * (r[i] - r[i - 1]) ** 2
                        - beta * (r[i] - r[i - 1]) ** 3
                        for i in range(N)
                    ]
                else:
                    a = [
                        k * (r[i + 1] - r[i])
                        + alpha * (r[i + 1] - r[i]) ** 2
                        + beta * (r[i + 1] - r[i]) ** 3
                        if i == 0
                        else -k * (r[i] - r[i - 1])
                        - alpha * (r[i] - r[i - 1]) ** 2
                        - beta * (r[i] - r[i - 1]) ** 3
                        if i == N - 1
                        else k * (r[i + 1] - r[i])
                        + alpha * (r[i + 1] - r[i]) ** 2
                        + beta * (r[i + 1] - r[i]) ** 3
                        - k * (r[i] - r[i - 1])
                        - alpha * (r[i] - r[i - 1]) ** 2
                        - beta * (r[i] - r[i - 1]) ** 3
                        for i in range(N)
                    ]

                r_temp = r
                r = [-r_0[i] + 2 * r[i] + a[i] * dt**2 for i in range(N)]
                v = [(r[i] - r_0[i]) / (2 * dt) for i in range(N)]
                r_k.append([dt * (i + 1), r])
                v_k.append([dt * (i + 1), v])
                r_0 = r_temp

        case "Verlet_v":
            pass
        case "Leapfrog":
            pass

    return r_k, v_k

In [8]:
N=int(18)
r0 = [1 if i==1 else 0 for i in range(N)]
v0 = [0]*N
rlist,vlist = FPU(18,1,0,0,r0,v0)
print(rlist)

[[0.01, [0.0001, 0.9998, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [0.02, [0.00029997, 0.99940006, 0.00029996000000000005, 1e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [0.03, [0.0005998500089999999, 0.9988002999810001, 0.0005998000150000001, 4.999400000000001e-08, 1e-12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [0.04, [0.0009995500629972, 0.9980008998670062, 0.0009994001049945002, 1.4995800280000002e-07, 6.9992e-12, 1e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [0.05, [0.001498950251974801, 0.9970020994680558, 0.0014986004199505022, 3.498320251988101e-07, 2.7992800450000004e-11, 8.998999999999999e-16, 1.0000000000000001e-20, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [0.06, [0.0020979007558740103, 0.995804198404279, 0.0020972012597525224, 6.994961259869108e-07, 8.396400494978102e-11, 4.498900066000001e-15, 1.09988e-19, 1.0000000000000001e-24, 0.0, 